<a href="https://colab.research.google.com/github/Sa1syo/NLTK/blob/main/IRNLP2020_Ex08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 7. Document Classification (Ch.6)

For International Students: goto http://www.nltk.org/book/ch06.html Almost corresponded about:

Lesson 1: 6.1 Supervised Classification (Exercise Attendance)  
Lesson 2: 6.2 Further Examples of Supervised Classification  
Lesson 3: 6.3 Evaluation  
Lesson 4: 6.4 Decision Trees  
(★ Assignment Remark): Please read carefully about 6.1. expecially 6.1.4, 6.1.6

Today's Topic:

- Some examples to utilize classification in NLTK
- Some examples of Supervised Classification
- How to make evaluation on each dataset

Please notice about each exmaple takes so long time to process in this exercise.

本日のトピック:

- NLTKに実装されているClassificationアルゴリズムの利用例
- その他Supervised Classificationの実行例
- データセットに対する評価尺度と手法

注意) この章辺りから、Document全体の計算をするので、1度1度のプロセスに時間がかかります。各回、終わるまで少々お待ちください。

この章の目的は、以下の質問に答えることです。

1. どのようにして分類する際に顕著な言語データの特定の特徴を特定することができますか？
2. どのようにして言語処理タスクを自動的に実行するために使用できる言語のモデルを構築することができますか？
3. これらのモデルから言語について何を学ぶことができますか？

In [ ]:
from __future__ import division  # Python 2 users only
import nltk, re, pprint
from nltk import word_tokenize

注意) この章辺りから、Document全体の計算をするので、1度1度のプロセスに時間がかかります。各回、終わるまで少々お待ちください。


## Lesson 1. Supervised Classification

Classification (分類) は、特定の入力に対して正しいクラスラベルを選択するタスクです。基本的な分類タスクでは、各入力は他のすべての入力から分離して考慮され、ラベルのセットは事前に定義されます。分類タスクの例を次に示します。

- メールがスパムかどうかを判断します。
- 「スポーツ」、「テクノロジー」、「政治」などのトピック領域の固定リストから、ニュース記事のトピックを決定します。
- 銀行という単語の特定の出現が、川岸、金融機関、側に傾く行為、または金融機関に何かを預ける行為を指すのに使用されるかどうかを決定します。

基本的な分類タスクには、いくつかの興味深い違いがあります。たとえば、マルチクラス分類では、各インスタンスに複数のラベルを割り当てることができます。オープンクラス分類では、ラベルのセットは事前に定義されていません。また、シーケンス分類では、入力のリストが一緒に分類されます。

分類器は、各入力の正しいラベルを含むコーパスのトレーニングに基づいて構築される場合、教師ありと呼ばれます。教師あり分類で使用されるフレームワークを以下に示します。


![image.png](attachment:image.png)



### 1.1. Gender Identification (Exercise 4で既に実施)

以下は既に行った、Naive Bayesによる名前からの性別の特定についての例です。

In [ ]:
nltk.download('names')
from nltk.corpus import names
import random

def gender_features(word):
    return {'last_letter': word[-1]}

labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
    [(name, 'female') for name in names.words('female.txt')])

random.shuffle(labeled_names)

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print('Neo: ', classifier.classify(gender_features('Neo')))
print('Trinity: ', classifier.classify(gender_features('Trinity')))
print('Overall: ', nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
Neo:  male
Trinity:  female
Overall:  0.738
Most Informative Features
             last_letter = 'k'              male : female =     43.1 : 1.0
             last_letter = 'a'            female : male   =     35.8 : 1.0
             last_letter = 'v'              male : female =     17.6 : 1.0
             last_letter = 'f'              male : female =     14.6 : 1.0
             last_letter = 'p'              male : female =     12.6 : 1.0


- 分類子を作成する最初のステップは、入力のどの 機能が関連するか、および それらの機能をエンコードする方法を決定することです。この例では、特定の名前の最後の文字を見ることから始めます。次の機能抽出 関数は、特定の名前に関する関連情報を含む辞書を作成します。

- フィーチャセットとして知られる返されたディクショナリは、フィーチャ名からその値にマッピングします。機能名は、大文字と小文字を区別する文字列で、通常は例の「last_letter」のように、人間が読み取れる機能の短い説明を提供します。フィーチャ値は、ブール値、数値、文字列などの単純なタイプの値です

- 最後のリストでは、「a」で終わるトレーニングセットの名前は男性よりも33倍多い女性ですが、「k」で終わる名前は女性より32倍多い男性であることを示しています。これらの比率は尤度比と呼ばれ、さまざまな機能と結果の関係を比較するのに役立ちます。

- 大きなコーパスを利用する場合は、全てのインスタンスの機能を含む単一のリストを作成すると、大量のメモリが消費される可能性があります。これらの場合、nltk.classify.applay_faeturesを利用します。これはリストのように利用できるが、全ての機能セットをメモリに保存せずにオブジェクトを返します。

In [ ]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])
classifier = nltk.NaiveBayesClassifier.train(train_set) # データがそのまま利用可能。

### 6.1.2. Choosing The Right Feature
特徴の選択とデータのエンコーディングの仕方の決定は、適した良いモデルを取る出す能力に非常に大きい影響を与えます。つまり、クラスタリング記述子を作成する際の多くの重要な作業は、関連する特徴の選定と表現能力の担保に他ならないのです。かなりシンプルで明らかな機能セットを使用することで適切なパフォーマンスを得ることができますが、通常は、目の前のタスクを十分に理解した上で、慎重に構築された特徴を使用することで大きなメリットが得られます。  
  
以下は、人名単語に対する性別を表す特徴を、試行錯誤のプロセスを経て構築する一例です。先ずは、全ての機能を含む『Kitchin Sink』アプローチから初めて、役立つ特徴を確認していきます。

In [ ]:
def gender_features2(name): # 最初と最後の文字に対して取得して調査。
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

print(gender_features2('John') )

{'first_letter': 'j', 'last_letter': 'n', 'count(a)': 0, 'has(a)': False, 'count(b)': 0, 'has(b)': False, 'count(c)': 0, 'has(c)': False, 'count(d)': 0, 'has(d)': False, 'count(e)': 0, 'has(e)': False, 'count(f)': 0, 'has(f)': False, 'count(g)': 0, 'has(g)': False, 'count(h)': 1, 'has(h)': True, 'count(i)': 0, 'has(i)': False, 'count(j)': 1, 'has(j)': True, 'count(k)': 0, 'has(k)': False, 'count(l)': 0, 'has(l)': False, 'count(m)': 0, 'has(m)': False, 'count(n)': 1, 'has(n)': True, 'count(o)': 1, 'has(o)': True, 'count(p)': 0, 'has(p)': False, 'count(q)': 0, 'has(q)': False, 'count(r)': 0, 'has(r)': False, 'count(s)': 0, 'has(s)': False, 'count(t)': 0, 'has(t)': False, 'count(u)': 0, 'has(u)': False, 'count(v)': 0, 'has(v)': False, 'count(w)': 0, 'has(w)': False, 'count(x)': 0, 'has(x)': False, 'count(y)': 0, 'has(y)': False, 'count(z)': 0, 'has(z)': False}


ただし、通常、特定の学習アルゴリズムで使用する特徴数には制限があります。提供する特徴が多すぎる場合、アルゴリズムは一般化されていないトレーニングデータの特異性に依存する可能性が高くなります。この問題は過剰適合として知られており、小さなトレーニングセットで作業する場合に特に問題になる可能性があります。  

たとえば、上記genger_feature2に示した特徴抽出器を使用して単純ベイズ分類器をトレーニングすると、比較的小さなトレーニングセットが過剰に適合し、その精度が注目される分類器の精度よりも約1％低いシステムになります。 (時々よくなります）

In [ ]:
random.shuffle(labeled_names)

featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.768
0.752


特徴の初期セットが選択されると、特徴セットを洗練する非常に生産的な方法はエラー分析です。最初に、モデルを作成するためのコーパスデータを含む開発セットを選択します。この開発セットは、トレーニングセットと開発テストセットに細分されます。

In [ ]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

Training Setはモデルのトレーニングに使用され、Dev-Test Setはエラー分析の実行に使用されます。Test Setは、システムの最終評価に役立ちます。以下で説明する理由により、Test Setを使用するだけでなく、エラー分析のために個別のDev-TestSetを使用することが重要です。コーパスデータの異なるサブセットへの分割を下図に示します。



![image.png](attachment:image.png)

それを用いて、Traingし、Dev-Testセットで比較します。

In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.761


(★ Assignment Remark)

Dev-Testセットを用いて、名前の性別を予測するときに、分類器が出力するエラーのリストを生成できます。
生成した後、モデルが間違ったラベルを予測した個々のエラーのケースを調べ、どの追加情報が正しい決定を可能にするか（または既存のどの情報がそれをだまして誤った決定を下すのか）を判断しようとします。

必要に応じて、特徴セットを調整できます。作成した名前分類器は、dev-testコーパスで約100個のエラーを生成します。

In [ ]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )
for (tag, guess, name) in sorted(errors[:20]): # [:20] は出力調整。外すと全てのリストが出ます。
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=April                         
correct=female   guess=male     name=Cris                          
correct=female   guess=male     name=Emmalyn                       
correct=female   guess=male     name=Janean                        
correct=female   guess=male     name=Kass                          
correct=female   guess=male     name=Katalin                       
correct=female   guess=male     name=Kipp                          
correct=female   guess=male     name=Nariko                        
correct=female   guess=male     name=Renel                         
correct=female   guess=male     name=Robin                         
correct=female   guess=male     name=Roslyn                        
correct=female   guess=male     name=Shaun                         
correct=male     guess=female   name=Avi                           
correct=male     guess=female   name=Barney                        
correct=male     guess=female   name=Bentley    

このエラーのリストを調べると、複数の文字のサフィックスが名前の性別を示している可能性があることが明らかになります。たとえば、ynで終わる名前は主に女性であるように見えますが、nで終わる名前は男性である傾向があります。また、hで終わる名前は 女性である傾向がありますが、chで終わる名前は通常男性です。したがって、2文字のサフィックスの機能を含めるように機能抽出ツールを調整します。

In [ ]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.777


# Exercise Attendance
上記では、2文字のSuffixを用いてFeatureとしましたが、3文字分抽出するとどうなるでしょうか？　1文字と2文字、3文字の時の精度を比較してください。
(ex)
    0.765
    0.771
    0.771

### 1.3. Document Classificatoin (Exercise 4で実施済み)

続いて、文書に対するクラスタリングを見ていきます。我々は文書がカテゴリで標識されているコーパスの例をいくつか見ました。これらのコーパスを使用して、新しいドキュメントに適切なカテゴリラベルを自動的にタグ付けする分類子を構築できます。まず、適切なカテゴリでラベル付けされたドキュメントのリストを作成します。この例では、各レビューをポジティブまたはネガティブに分類するMovie Reviews Corpusを選択しました。


In [ ]:
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
           for category in movie_reviews.categories()
           for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [ ]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

0.82


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
    contains(schumacher) = True              neg : pos    =     11.8 : 1.0
        contains(welles) = True              neg : pos    =      7.8 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.8 : 1.0
          contains(mena) = True              neg : pos    =      7.1 : 1.0
     contains(atrocious) = True              neg : pos    =      7.1 : 1.0


### 6.1.4. Part-of-Speech Tagging

(★ Assignment Remark)

Exercise 5及び6で、我々は言葉の内部の構造を見て、単語の品詞タグを選択する正規表現タガーを構築しました。ただし、この正規表現タガーは手作りする必要がありました。代わりに、どのサフィックスが最も有益かを判断するために分類器をトレーニングできます。最も一般的なサフィックスが何であるかを調べることから始めましょう。

In [ ]:
nltk.download('brown')
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1
    
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


これのSuffixから、特定の単語をチェックする特徴抽出関数を定義します。

In [ ]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

この特徴抽出関数はFilterとなり、データの一部のプロパティを強調し、他のプロパティを表示できないようにします。分類器は、入力にラベルを付ける方法を決定するときに、この強調されたプロパティのみを用いて分類します。

続いて、この特徴分類器を用いて、Decision Tree分類器をトレーニングします。（Lesson 4で説明します）
(注意: 非常にトレーニングに時間が掛かります。）

In [ ]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
print(classifier.classify(pos_features('cats')))
print(classifier.pseudocode(depth=4))

0.6270512182993535
NNS
if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



Decision Treeモデルの優れた機能の1つは、それらの解釈が非常に簡単な場合が多いことです。NLTKに疑似コードとして出力するように指示することもできます。

ここでは、単語がTheで終わるかどうかを確認します。その場合、ほぼ確実に決定子になります。  
単語「the」は非常に一般的であるため、この「接尾辞」は決定ツリーで早期に使用されます。  
次に、カンマで終わるかどうかを確認します。カンマで終わる場合、特別なタグ","を受け取ります。  
続いて、分類子は単語が「s」で終わるかどうかをチェックします。  
その場合、PrepositionタグPP$を受け取る可能性が最も高く（特別なタグBEZを持つ「is」という単語でない限り）、そうでない場合は、あーてぃくyレーション（句読点 "。"でない限り）である可能性が最も高いです。  
引数は、決定木の上部のみを表示します。

### 1.5. Exploiting Context

特徴抽出機能を強化することにより、この品詞タガーを変更して、単語の長さ、含まれる音節の数、またはその接頭辞など、他のさまざまな単語内部機能を活用できます。ただし、機能抽出プログラムが対象の単語を見る限り、その単語が表示されるコンテキストに依存する機能を追加する方法はありません。ただし、コンテキスト機能は、正しいタグに関する強力な手がかりを提供します。 「fly」という単語は、前の単語が「a」であることを知っているため、動詞ではなく名詞として機能していると判断できます。

単語のコンテキストに依存する機能に対応するには、機能抽出ツールの定義に使用したパターンを修正する必要があります。タグ付けする単語を単に渡すのではなく、ターゲットワードのインデックスとともに、完全な（タグなし）文を渡します。このアプローチは、コンテキスト依存の特徴抽出機能を使用して音声タグ分類子を定義する以下のコードで実証されています。

In [ ]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

print(pos_features(brown.sents()[0], 8))

tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag) )

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}
0.7891596220785678


コンテキスト機能を活用すると、品詞タガーのパフォーマンスが向上することは明らかです。たとえば、分類子は、単語 "large"または単語 "gubernatorial"の直後に単語がある場合、その単語は名詞である可能性が高いことを学習します。ただし、形容詞に続く場合、単語はおそらく名詞であるという一般化を学習することはできません。前の単語の品詞タグにアクセスできないためです。  

一般に、単純な分類器は、各入力を常に他のすべての入力から独立したものとして扱います。多くの状況で、これは完全に理にかなっています。たとえば、名前が男性であるか女性であるかについての決定は、ケースバイケースで行うことができます。

### 1.6. Sequence Classification

(★ Assignment Remark)

関連する分類タスク間の依存関係をキャプチャするために、関連する入力のコレクションに適切なラベル付けを選択する共同分類器モデルを使用できます。品詞タグ付けの場合、さまざまな異なるシーケンス分類器モデルを使用して、特定の文のすべての単語に対して品詞タグを共同で選択できます。

連続分類または貪欲なシーケンス分類として知られるシーケンス分類戦略の1つは、最初の入力で最も可能性の高いクラスラベルを見つけ、その回答を使用して次の入力で最適なラベルを見つけることです。  
すべての入力にラベルが付けられるまで、プロセスを繰り返すことができます。これは、Lesson 6のバイグラムタガーによって採用されたアプローチです。これは、文の最初の単語の品詞タグを選択することから始まり、前の単語自体と予測に基づいて後続の各単語のタグを選択しました。

この戦略は下記のコードで実証されています。まず、特徴抽出関数を拡張してHistory引数を取得する必要があります。  
History引数は、これまでの文に対して予測したタグのリストを提供します。各タグの歴史は、内の単語に対応する文。ただし、Historyには、既に分類した単語、つまりターゲット単語の左側の単語のタグのみが含まれることに注意してください。したがって、ターゲットワードの右側にあるワードの一部の機能を調べることは可能ですが、それらのワードのタグを調べることはできません。

特徴抽出機能を定義したら、シーケンス分類子の作成に進むことができます。トレーニング中に、注釈付きタグを使用して適切なHistoryを機能抽出器に提供しますが、新しい文にタグを付けるときは、タガー自体の出力に基づいてHistoryリストを生成します。

In [ ]:
def pos_features(sentence, i, history): 
     features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
     if i == 0:
         features["prev-word"] = "<START>"
         features["prev-tag"] = "<START>"
     else:
         features["prev-word"] = sentence[i-1]
         features["prev-tag"] = history[i-1]
     return features

class ConsecutivePosTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

(注意: タスクに時間が掛かります）

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

0.7980528511821975


## Lesson 2. Further Examples of Supervised Classification

### 2.1. Sentence Segmentation

※ Assignment Remark

文のセグメンテーションは、句読点の分類タスクと見なすことができます。ピリオドや疑問符など、文を終了させる可能性のある記号に出会うたびに、前の文を終了するかどうかを判断する必要があります。

最初のステップは、すでに文にセグメント化されているいくつかのデータを取得し、それを特徴の抽出に適した形式に変換することです。

ここで、トークンは個々の文からのトークンのマージされたリストであり、境界はすべての文境界トークンのインデックスを含むセットです。次に、句読点が文の境界を示しているかどうかを判断するために使用されるデータの特徴を指定する必要があります。

In [ ]:
nltk.download('treebank')
sents = nltk.corpus.treebank_raw.sents()

def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
            'prev-word': tokens[i-1].lower(),
            'punct': tokens[i],
            'prev-word-is-one-char': len(tokens[i-1]) == 1}

tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)


featuresets = [(punct_features(tokens, i), (i in boundaries))
               for i in range(1, len(tokens)-1)
               if tokens[i] in '.?!']

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
0.936026936026936


この分類子を使用して文のセグメンテーションを実行するには、各句読点をチェックして、境界としてラベル付けされているかどうかを確認します。境界マークで単語のリストを分割します。以下のコードは、これを行う方法を示しています。

In [ ]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if i != len(words)-1 and word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents


対話を処理するとき、発話を話者によって実行されるアクションの一種と考えると便利です。この解釈は、「私はあなたを許します」または「あなたはその丘に登ることはできないに違いない」などのパフォーマンスのあるステートメントに対して最も簡単です。ただし、挨拶、質問、回答、主張、説明はすべて、音声ベースのアクションの一種と考えることができます。対話の発話の根底にある対話行為を認識する ことは、会話を理解する上で重要な最初のステップになります。

NPSチャットコーパスは、 インスタントメッセージングセッションからの10,000を超える投稿で構成されています。これらの投稿にはすべて、「Statement」、「Emotion」、「ynQuestion」、「Continuer」などの15種類の対話行為のいずれかのラベルが付いています。したがって、このデータを使用して、新しいインスタントメッセージング投稿の対話行為の種類を識別することができる分類子を構築できます。最初のステップは、基本的なメッセージングデータを抽出することです。xml_posts（）を呼び出して、各投稿のXML注釈を表すデータ構造を取得します。

In [ ]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


次に、投稿に含まれる単語をチェックする簡単な特徴抽出機能を定義します.

In [ ]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

最後に、各投稿に特徴抽出機能を適用して（post.get（'class'）を使用して投稿の対話行為タイプを取得する）トレーニングおよびテストデータを構築し、新しい分類子を作成します。

In [ ]:
nltk.download('punkt')
featuresets = [(dialogue_act_features(post.text), post.get('class'))
               for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0.668


### 2.3. テキストの含意の認識

テキスト含意（RTE）の認識は、特定のテキストTが「仮説」と呼ばれる別のテキストを含意するかどうかを決定するタスクです。これまでに、4つのRTEチャレンジがあり、競合するチームが開発とテストの共有データを利用できるようになりました。以下は、チャレンジ3開発データセットのテキスト/仮説のペアの例です。ラベルTrueは含意が成り立つことを示し、Falseは含意が成り立たないことを示します。

チャレンジ3、ペア34（True）

T：パルビズダブディは、ロシア、中国、中央アジアの4つの旧ソビエト連邦を結びつけてテロと戦う、駆け出しの協会である上海協力機構（SCO）の会議でイランを代表していました。

H：中国はSCOのメンバーです。

チャレンジ3、ペア81（False）

T：NCの定款によれば、LLC会社のメンバーはH.ネルソンビーバーズ、III、H。チェスタービーバーズ、ジェニービーバーズスチュワートです。

H：Jennie Beavers Stewartは、Carolina Analytical Laboratoryの株主です。

テキストと仮説の関係は論理的含意を意図するものではなく、むしろ、テキストが仮説を真にするための合理的な証拠を提供すると人間が結論付けるかどうかを強調する必要があります。

RTEを分類タスクとして扱うことができます。このタスクでは、各ペアのTrue / Falseラベルを予測しようとします。このタスクに対する成功したアプローチには、構文解析、セマンティクス、および実世界の知識の組み合わせが含まれる可能性が高いようですが、RTEの初期の試みの多くは、テキストと単語レベルでの仮説の類似性に基づいて、浅い分析でかなり良い結果を達成しました 理想的なケースでは、含意がある場合、仮説によって表されるすべての情報もテキストに存在するはずです。逆に、テキストにない情報が仮説で見つかった場合、含意はありません。

以下のRTE特徴検出器では、単語（つまり、単語の種類）を情報のプロキシとして機能させ、特徴は単語の重複の程度と、仮説にはあるがテキストにはない単語の数をカウントします（メソッドhyp_extra（）によってキャプチャされます）。  
すべての単語が同じように重要であるわけではありません。名前付きエンティティの言及は、人、組織、場所などの名前がより重要である可能性が高いため、単語 sとne（名前付きエンティティ）の個別の情報を抽出するように動機付けられます。  
さらに、いくつかの高周波機能ワードは、「ストップワード」として除外されます

In [ ]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap'] = len(extractor.overlap('word'))
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

これらの機能の内容を説明するために、前に示したテキスト/仮説ペア34のいくつかの属性を調べます。

In [ ]:
nltk.download('rte')
rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)

[nltk_data] Downloading package rte to /root/nltk_data...
[nltk_data]   Package rte is already up-to-date!
{'republics', 'binds', 'that', 'at', 'operation', 'meeting', 'China', 'Iran', 'Co', 'Asia', 'Soviet', 'representing', 'former', 'fledgling', 'Russia', 'association', 'Organisation', 'was', 'terrorism.', 'Davudi', 'SCO', 'fight', 'Parviz', 'Shanghai', 'together', 'four', 'central'}


In [ ]:
 print(extractor.hyp_words)

{'member', 'SCO.', 'China'}


In [ ]:
 print(extractor.overlap('word'))

set()


In [ ]:
print(extractor.overlap('ne'))

{'China'}


In [ ]:
print(extractor.hyp_extra('word'))

{'member'}


### 2.4. 大規模データセットへのスケールアップ

Pythonは、基本的なテキスト処理と特徴抽出を実行するための優れた環境を提供します。ただし、機械学習方法に必要な数値集中計算をCなどの低レベル言語とほぼ同じ速度で実行することはできないとされてきました。したがって、純粋なPython機械学習実装（nltk.NaiveBayesClassifierなど）を使用しようとすると、大規模なデータセットでは、学習アルゴリズムの完了に不合理な時間とメモリがかかることがあります。  
しかしながら、近年では、Pythonテクノロジの向上により、C言語等よりもより早く、GPU並列化等を組み込むことが出来るライブラリが多数存在する上に、JIT（Just in time)コンパイラの組み込みにより、C言語と遜色ない速度で検証することが出来るようになってきました。このため、ディープラーニング等の計算量の高いアルゴリズムへの検証実装等は、ほぼPythonの独擅場となってきています。

大量のトレーニングデータまたは多数の機能を使用して分類器をトレーニングする予定がある場合は、外部の機械学習パッケージとインターフェイスするためのNLTKの機能を検討することをお勧めします。これらのパッケージがインストールされると、NLTKはそれらを（システムコールを介して）透過的に呼び出して、純粋なPython分類子の実装よりもはるかに高速に分類子モデルをトレーニングできます。NLTKでサポートされている推奨機械学習パッケージのリストについては、NLTK Webページを参照してください。
また、機械学習を行う場合は、Scikit-Learn(https://scikit-learn.org/) 等を使うことをお勧めします。

## Lesson 3. Evaluation
分類モデルがパターンを正確にキャプチャしているかどうかを判断するには、そのモデルを評価する必要があります。この評価の結果は、モデルの信頼性を決定するために、そしてどのような目的でモデルを使用できるかを決定するために重要です。評価は、モデルの将来の改善を行う上で私たちを導くための効果的なツールにもなります

### 3.1. Test Set

ほとんどの評価手法は、テストセット （または評価セット）の入力に対して生成されたラベルを、それらの入力の正しいラベルと比較することにより、モデルのスコアを計算します。通常、このテストセットの形式はトレーニングセットと同じです。ただし、テストセットがトレーニングコーパスとは異なることは非常に重要です。トレーニングセットをテストセットとして単純に再利用する場合、新しいサンプルに一般化する方法を学習せずに、入力を単純に記憶するモデルは、誤解を招くほど高いスコアを受け取ります。

テストセットを作成するとき、テストに使用できるデータの量とトレーニングに使用できる量の間にトレードオフがしばしばあります。少数のバランスのとれたラベルと多様なテストセットを持つ分類タスクの場合、わずか100の評価インスタンスで意味のある評価を実行できます。  
ただし、分類タスクに多数のラベルがある場合、または非常にまれなラベルが含まれている場合は、最も頻度の低いラベルが少なくとも50回発生するようにテストセットのサイズを選択する必要があります。  
さらに、テストセットに1つのドキュメントから描画されたインスタンスなど、密接に関連する多くのインスタンスが含まれる場合、テストセットのサイズを大きくして、この多様性の欠如が評価結果を歪めないようにする必要があります。

テストセットを選択する際のもう1つの考慮事項は、テストセットのインスタンスと開発セットのインスタンスの類似度です。これら2つのデータセットが類似しているほど、評価結果が他のデータセットに一般化されるという確信が低くなります。たとえば、品詞タグ付けタスクについて考えてみましょう。極端な場合、単一のジャンル（ニュース）を反映するデータソースから文をランダムに割り当てることにより、トレーニングセットとテストセットを作成できます。

In [ ]:
import random
from nltk.corpus import brown
tagged_sents = list(brown.tagged_sents(categories='news'))
random.shuffle(tagged_sents)
size = int(len(tagged_sents) * 0.1)
train_set, test_set = tagged_sents[size:], tagged_sents[:size]

この場合、テストセットはトレーニングセットと非常によく似ています。トレーニングセットとテストセットは同じジャンルから取得されるため、評価結果が他のジャンルに一般化されるとは確信できません。さらに悪いことに、random.shuffle（）の呼び出しのために、 テストセットにはトレーニングに使用された同じドキュメントから取得された文が含まれています。文書内に一貫したパターンがある場合（たとえば、特定の品詞タグが特に頻繁に出現する場合など）、その違いは開発セットとテストセットの両方に反映されます。やや優れたアプローチは、トレーニングセットとテストセットが異なるドキュメントから取得されるようにすることです。

In [ ]:
file_ids = brown.fileids(categories='news')
size = int(len(file_ids) * 0.1)
train_set = brown.tagged_sents(file_ids[size:])
test_set = brown.tagged_sents(file_ids[:size])

In [ ]:
train_set = brown.tagged_sents(categories='news')
test_set = brown.tagged_sents(categories='fiction')

### 3.2. Accuracy

分類器を評価するために使用することができる最も簡単なメトリック、 精度、分類器が正しくラベルされたことをテストセットでの入力の割合を測定します。たとえば、80個の名前を含むテストセットで正しい名前を60回予測する名前性別分類子の精度は60/80 = 75％です。関数nltk.classify.accuracy（）は、特定のテストセットで分類子モデルの精度を計算します。

    classifier = nltk.NaiveBayesClassifier.train(train_set) f
    print(nltk.classify.accuracy(classifier,test_set))
    print('Accuracy: {:4.2f}'.format(nltk.classify.accuracy(classifier, test_set))) 
    
    0.75
    
分類子の精度スコアを解釈するときは、テストセット内の個々のクラスラベルの頻度を考慮することが重要です。  
たとえば、単語bankが出現するたびに正しい単語の意味を決定する分類器を考えます。Wired Bank Newsのテキストでこの分類器を評価すると、金融機関のsenseが20回のうち19回現れることがあります。その場合、95％の精度はモデルでその精度を達成できるため、ほとんど印象的ではありません。それは常に金融機関をsenseとして返します。  
ただし、代わりに、最も頻度の高い単語の意味が40％の頻度である、よりバランスのとれたコーパスで分類器を評価する場合、95％の精度スコアがはるかに良い結果になります。

### 3.3. Precision and Recall

精度スコアが誤解を招く可能性がある別の例は、特定のタスクに関連するドキュメントの検索を試みる情報検索などの「検索」タスクです。無関係なドキュメントの数は、関連するドキュメントの数をはるかに上回るため、すべてのドキュメントを無関係とラベル付けするモデルの精度スコアは、100％に非常に近くなります。

![image.png](attachment:image.png)

したがって、上図に示されている4つのカテゴリのそれぞれのアイテムの数に基づいて、検索タスクに異なるメジャーセットを採用するのが一般的です。

 - True-Positive (真陽性) は、関連性があると正しく識別された関連項目です。
 - True-Negative (真陰性) は、無関係であると正しく識別した無関係なアイテムです。
 - False-Positive (偽陽性) 、誤検知（またはタイプIエラー）は、関連性があると間違って診断した関連性のないアイテムです。
 - False-Nevative (偽陰性、またはタイプIIエラー）は、関連性があるのに不適切であると誤って特定した関連項目です。
 
これらの4つの数値が与えられると、次のメトリックを定義できます。

- Precision (精度) は、特定したアイテムのうちどれだけが関連していたかを示すもので、TP /（TP + FP）です。
- Recall (再現率) は、特定した関連アイテムの数がTP /（TP + FN）であることを示しています。
- F-Measure (F値)（またはF-スコア（単一のスコアを与えるための精度と再現率を兼ね備え）、精度と再現率の調和平均であると定義される.　  
(2 x 精度 x リコール）/（精度 + リコール）。

### 3.4. Confusion Matrix

3つ以上のラベルを使用して分類タスクを実行する場合、どのタイプの間違いに基づいてモデルによって行われたエラーを再分割することが有益である可能性があります。混同行列は、各セル(表であるi、j)はラベル頻度を示し、jが正しいラベルとした場合に、iが予測され、その数や割合をカウントする。したがって、対角エントリ（つまり、セル| ii |）は正しく予測されたラベルを示し、非対角エントリはエラーを示します。次の例では、Lesson 5で開発されたバイグラムタガーの混同行列を生成します。

In [ ]:
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t2.evaluate(test_sents)

0.8723226703755216

In [ ]:
def tag_list(tagged_sents):
    return [tag for sent in tagged_sents for (word, tag) in sent]
def apply_tagger(tagger, corpus):
    return [tagger.tag(nltk.tag.untag(sent)) for sent in corpus]
gold = tag_list(brown.tagged_sents(categories='editorial'))
test = tag_list(apply_tagger(t2, brown.tagged_sents(categories='editorial')))
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

    |                                         N                      |
    |      N      I      A      J             N             V      N |
    |      N      N      T      J      .      S      ,      B      P |
----+----------------------------------------------------------------+
 NN | <11.9%>  0.0%      .   0.2%      .   0.0%      .   0.2%   0.0% |
 IN |   0.0%  <9.0%>     .      .      .   0.0%      .      .      . |
 AT |      .      .  <8.6%>     .      .      .      .      .      . |
 JJ |   1.7%      .      .  <4.0%>     .      .      .   0.0%   0.0% |
  . |      .      .      .      .  <4.8%>     .      .      .      . |
NNS |   1.4%      .      .      .      .  <3.3%>     .      .   0.0% |
  , |      .      .      .      .      .      .  <4.4%>     .      . |
 VB |   1.0%      .      .   0.0%      .      .      .  <2.4%>     . |
 NP |   1.0%      .      .   0.0%      .      .      .      .  <1.9%>|
----+----------------------------------------------------------------+
(row =

### 3.5. Cross Validation

モデルを評価するには、注釈付きデータの一部をテストセット用に予約する必要があります。すでに述べたように、テストセットが小さすぎる場合、評価は正確ではない可能性があります。ただし、テストセットを大きくすることは、通常、トレーニングセットを小さくすることを意味し、限られた量の注釈付きデータが利用可能な場合、パフォーマンスに大きな影響を与える可能性があります。

この問題の解決策の1つは、異なるテストセットで複数の評価を実行し、それらの評価から得られたスコアを相互検証と呼ばれる手法で結合することです。特に、元のコーパスをfolds と呼ばれるN個のサブセットに分割します。これらの各フォールドについて、そのフォールド内のデータを除くすべてのデータを使用してモデルをトレーニングし、フォールド上でそのモデルをテストします。個々のフォールドが小さすぎて正確な評価スコアを単独で提供できない場合でも、複合評価スコアは大量のデータに基づいているため、非常に信頼性が高くなります。

クロスバリデーションを使用するもう1つの重要な利点は、異なるトレーニングセット間でパフォーマンスがどれほど大きく変化するかを調べることができることです。N個すべてのトレーニングセットについて非常に類似したスコアを取得する場合 、スコアが正確であるとかなり確信で​​きます。一方、N個のトレーニングセットでスコアが大きく異なる場合は、評価スコアの精度についておそらく懐疑的です。

## Lesson 4. Desicion Tree

決定木は、単純なフローチャートである入力値の選択ラベルこと。このフローチャートは、特徴値をチェックする決定ノードと、ラベルを割り当てるリーフノードで構成されています。入力値のラベルを選択するには、ルートノードと呼ばれるフローチャートの初期決定ノードから始めます。このノードには、入力値の機能の1つをチェックし、その機能の値に基づいてブランチを選択する条件が含まれています。入力値を説明するブランチに続いて、入力値の機能に新しい条件を設定した新しい決定ノードに到達します。入力値のラベルを提供するリーフノードに到達するまで、各ノードの条件によって選択されたブランチを追跡し続けます。 以下の図は 名前性別タスクのデシジョンツリーモデルの例を示します。
![image.png](attachment:image.png)

デシジョンツリーを作成したら、それを使用して新しい入力値にラベルを割り当てるのは簡単です。それほど簡単ではないのは、特定のトレーニングセットをモデル化する決定木を構築する方法です。しかし、意思決定ツリーを構築するための学習アルゴリズムを検討する前に、コーパスに最適な「意思決定の切り株」を選択する、より単純なタスクを検討します。意思決定の切り株は、単一の機能に基づいて入力を分類する方法を決定する単一のノードを持つ決定ツリーです。可能なフィーチャ値ごとに1つのリーフが含まれ、その値を持つフィーチャの入力に割り当てられるクラスラベルを指定します。決定の切り株を作成するには、最初にどの機能を使用するかを決定する必要があります。最も簡単な方法は、可能な機能ごとに意思決定の切り株を作成し、トレーニングデータでどれが最高の精度を達成するかを確認することです。ただし、以下で説明する他の選択肢もあります。機能を選択したら、トレーニングセット内の選択した例（選択した機能にその値がある例）の最も頻繁なラベルに基づいて各リーフにラベルを割り当てることにより、決定スタンプを作成できます。

決定切り株を選択するためのアルゴリズムを考えると、より大きな決定木を成長させるためのアルゴリズムは簡単です。まず、分類タスクの全体的な最適な決定スタンプを選択することから始めます。次に、トレーニングセットの各リーフの精度をチェックします。十分な精度を達成できない葉は、葉へのパスによって選択されたトレーニングコーパスのサブセットでトレーニングされた新しい決定切り株に置き換えられます。たとえば、「k」で始まらず、母音または「l」で終わらないトレーニングセット名のサブセットでトレーニングされた新しい左端の葉を新しい決定切り株に置き換えることにより、4.1 の決定木を成長させることができます。 」

### 4.1. Entropy and Information Gain

前述したように、意思決定の切り株の最も有益な機能を識別するための方法がいくつかあります。情報ゲインと呼ばれる人気のある代替手段は、特定の機能を使用して入力値を分割したときに、入力値がどれだけ整理されるかを測定します。入力値の元のセットがどの程度無秩序であるかを測定するために、ラベルのエントロピーを計算します。これは、入力値に非常に多様なラベルがある場合は高く、多くの入力値がすべて同じラベルを持つ場合は低くなります。特に、エントロピーは、各ラベルの確率の合計にその同じラベルの対数確率を掛けたものとして定義されます。

![image.png](attachment:image.png)

以下のコードは、ラベルのリストのエントロピーを計算する方法です。

In [ ]:
import math
def entropy(labels):
    freqdist = nltk.FreqDist(labels)
    probs = [freqdist.freq(l) for l in freqdist]
    return -sum(p * math.log(p,2) for p in probs)

In [ ]:
print(entropy(['male', 'male', 'male', 'male'])) 
print(entropy(['male', 'female', 'male', 'male']))
print(entropy(['female', 'male', 'female', 'male']))
print(entropy(['female', 'female', 'male', 'female']))
print(entropy(['female', 'female', 'female', 'female'])) 

-0.0
0.8112781244591328
1.0
0.8112781244591328
-0.0


入力値の元のラベルのセットのエントロピーを計算したら、決定スタンプを適用すると、ラベルがどの程度整理されるかを決定できます。そのために、決定切り株の各葉のエントロピーを計算し、それらの葉のエントロピー値の平均（各葉のサンプル数で重み付け）を取得します。情報ゲインは、元のエントロピーからこの新しい減少したエントロピーを引いたものに等しくなります。情報ゲインが高いほど、入力値をコヒーレントグループに分割する決定スタンプの仕事がより良くなるため、情報ゲインが最も高い決定スタンプを選択することで決定木を構築できます。

決定木のもう1つの考慮事項は効率です。上記の決定スタンプを選択するための簡単なアルゴリズムは、考えられるすべての機能に対して候補決定スタンプを構築する必要があり、構築された決定ツリー内のすべてのノードに対してこのプロセスを繰り返す必要があります。以前に評価された例に関する情報を保存および再利用することにより、トレーニング時間を短縮するための多くのアルゴリズムが開発されました。

決定木には多くの有用な性質があります。そもそも、理解しやすく、解釈しやすいものです。これは、意思決定ツリーの最上部付近で特に当てはまります。通常、学習アルゴリズムは非常に有用な機能を見つけることができます。決定木は、多くの階層的なカテゴリの区別を行うことができる場合に特に適しています。たとえば、決定木は系統樹のキャプチャに非常に効果的です。

ただし、決定木にはいくつかの欠点もあります。1つの問題は、決定ツリーの各ブランチがトレーニングデータを分割するため、ツリーの下位ノードをトレーニングするために使用できるトレーニングデータの量が非常に少なくなる可能性があることです。その結果、これらの下位決定ノードは

トレーニングセットをオーバーフィットし、基礎となる問題の言語的に重要なパターンではなく、トレーニングセットの特異性を反映する学習パターン。この問題の1つの解決策は、トレーニングデータの量が少なくなりすぎるとノードの分割を停止することです。別の解決策は、完全な決定ツリーを成長させることですが、その後、開発テストのパフォーマンスを改善しない決定ノードを 除去することです。

デシジョンツリーの2番目の問題は、フィーチャが互いに比較的独立して動作する場合でも、フィーチャを特定の順序で強制的にチェックすることです。たとえば、ドキュメントをトピック（スポーツ、自動車、殺人ミステリーなど）に分類する場合、hasword（football）などの機能は、他の機能値に関係なく、特定のラベルを非常に示します。デシジョンツリーの上部付近にはスペースが限られているため、これらの機能のほとんどは、ツリー内のさまざまなブランチで繰り返す必要があります。そして、ツリーを下るにつれて分岐の数が指数関数的に増加するため、繰り返しの量は非常に大きくなる可能性があります。

関連する問題は、ディシジョンツリーが、正しいラベルの弱い予測子である機能の使用に適していないことです。これらの機能は比較的小さな漸進的な改善を行うため、決定ツリーで非常に低い頻度で発生する傾向があります。しかし、決定木学習者がこれらの機能を使用するのに十分なほど下降した時点では、どのような効果があるかを確実に判断するのに十分なトレーニングデータが残っていません。代わりに、トレーニングセット全体でこれらの機能の効果を見ることができれば、ラベルの選択にどのように影響するかについて結論を出すことができるかもしれません。

ディシジョンツリーでは、特定の順序で機能をチェックする必要があるため、比較的独立した機能を活用する能力が制限されます。次に説明する単純なベイズ分類法は、すべての機能を「並行して」動作させることにより、この制限を克服します。